# Day 20
## Part 1

In [56]:
import parse
import itertools
from collections import defaultdict
from pyrsistent import pvector, pset
from functools import lru_cache
import operator
from functools import reduce
import re

def parse_data(s):
    lines = s.strip().splitlines()
    tiles = {}
    for i, line in enumerate(lines):
        if (p := parse.parse('Tile {id:d}:', line)):
            tiles[p['id']] = pvector(lines[i + 1: i + 11])
            
    return tiles

In [57]:
test_input = open('test_input').read()
test_tiles = parse_data(test_input)

In [58]:
len(test_tiles) ** 0.5

3.0

In [59]:
def north_edge(tile):
    return tile[0]

def south_edge(tile):
    return tile[-1]

def east_edge(tile):
    return ''.join(row[-1] for row in tile)

def west_edge(tile):
    return ''.join(row[0] for row in tile)

def possible_edges(tile):
    edges = {north_edge(tile), south_edge(tile),
             west_edge(tile), east_edge(tile)}
    return edges | {''.join(reversed(e)) for e in edges}

In [60]:
data = open('input').read()
tiles = parse_data(data)

In [61]:
def generate_graph(tiles):
    g = defaultdict(set)
    for t1, t2 in itertools.combinations(tiles, 2):
        if possible_edges(tiles[t1]) & possible_edges(tiles[t2]):
            g[t1].add(t2)
            g[t2].add(t1)
    return g

In [62]:
generate_graph(test_tiles)

defaultdict(set,
            {2311: {1427, 1951, 3079},
             1951: {2311, 2729},
             1427: {1489, 2311, 2473, 2729},
             3079: {2311, 2473},
             2729: {1427, 1951, 2971},
             1171: {1489, 2473},
             1489: {1171, 1427, 2971},
             2473: {1171, 1427, 3079},
             2971: {1489, 2729}})

In [63]:
g = generate_graph(tiles)

In [64]:
[n for n in g if len(g[n]) == 2]

[1699, 2351, 1433, 3229]

In [65]:
import math
math.prod(_)

18482479935793

## Part 2
I can't say I wasn't warned.

In [66]:
len(g) ** 0.5

12.0

In [67]:
len([n for n in g if len(g[n]) == 3])/4

10.0

Ok, so it's a 12 by 12 square. Let's map it out starting from one of the corners.

In [68]:
def map_square(graph):
    # Width of square
    d = int(len(graph) ** 0.5)
    # Pick one of the corners
    corner = {t for t in graph if len(graph[t]) == 2}.pop()
    assigned = {corner}
    unassigned = set(graph.keys()) - {corner}
    square = {(0, 0): corner}
    
    for x in range(1, d - 1):
        # Add a neighbour if it has a degree of 3, therefore
        # is on the square's edge
        next_tile = ({t for t in graph[square[(x - 1, 0)]] 
                      if len(graph[t]) == 3} & unassigned).pop()
        square[(x, 0)] = next_tile
        assigned.add(next_tile)
        unassigned.discard(next_tile)
        
    # Find the top right corner
    next_tile = ({t for t in graph[square[(d - 2, 0)]] 
                 if len(graph[t]) == 2} & unassigned).pop()
    assigned.add(next_tile)
    unassigned.discard(next_tile)
    square[(d - 1, 0)] = next_tile
    
    for y in range(1, d - 1):
        # Add a neighbour if it has a degree of 3, therefore
        # is on the square's edge
        next_tile = ({t for t in graph[square[(0, y - 1)]] 
                      if len(graph[t]) == 3} & unassigned).pop()
        square[(0, y)] = next_tile
        assigned.add(next_tile)
        unassigned.discard(next_tile)
        
    # Find the bottom left corner
    next_tile = ({t for t in graph[square[(0, d - 2)]] 
                 if len(graph[t]) == 2} & unassigned).pop()
    assigned.add(next_tile)
    unassigned.discard(next_tile)
    square[(0, d - 1)] = next_tile
    
    # Now add the rest from their top and left corners
    for x in range(1, d):
        for y in range(1, d):
#             print(square)
#             print(x, y)
#             print(graph[square[(x - 1, y)]])
#             print(graph[square[(x, y - 1)]])
#             print(unassigned)
            next_tile = (graph[square[(x - 1, y)]] 
                         & graph[square[(x, y - 1)]]
                         & unassigned).pop()
            square[(x, y)] = next_tile
            assigned.add(next_tile)
            unassigned.discard(next_tile)

    return square

In [69]:
@lru_cache(maxsize=None)
def flips_and_rotates(tile):
    d = len(tile)
    result = pset()
    result = result.add(tile)
    # flip vertically
    result = result.add(pvector(reversed(tile)))
    # flip horizontally
    result = result.add(pvector(''.join(reversed(row)) for row in tile))
    # rotate by 90 degrees three times and flip
    # - some of this will be redundant
    for _ in range(3):
        tile = pvector([
            ''.join(reversed(list(tile[i][j] for i in range(d))))
            for j in range(d)
        ])
        result = result.add(tile)
        result = result.add(pvector(reversed(tile)))
        result = result.add(pvector(''.join(reversed(row)) for row in tile))
    return result

my_image = pvector('''abcde
fghij
klmno
pqrst
uvwxy'''.splitlines())

for transform in set(flips_and_rotates(my_image)):
    print('\n'.join(transform))
    print()

yxwvu
tsrqp
onmlk
jihgf
edcba

abcde
fghij
klmno
pqrst
uvwxy

edcba
jihgf
onmlk
tsrqp
yxwvu

uvwxy
pqrst
klmno
fghij
abcde

ejoty
dinsx
chmrw
bglqv
afkpu

upkfa
vqlgb
wrmhc
xsnid
ytoje

ytoje
xsnid
wrmhc
vqlgb
upkfa

afkpu
bglqv
chmrw
dinsx
ejoty



I'm going to hope that the neighbouring edges are unique as there are about $2^9$ of them, accounting for reversals, so I'm not going to search through different permutations of rotations etc.

In [70]:
def orient_tiles(tiles, square):
    sea = {}
    top_left = tiles[square[(0, 0)]]
    neighbour = tiles[square[(1, 0)]]
    lower_neighbour = tiles[square[(0, 1)]]
    c = itertools.count(1)
    # Orient these three correctly, then the rest
    # will follow
    for t1, t2, t3 in itertools.product(
        flips_and_rotates(top_left), 
        flips_and_rotates(neighbour),
        flips_and_rotates(lower_neighbour)
    ):
        if east_edge(t1) == west_edge(t2) and south_edge(t1) == north_edge(t3):
            sea[(0, 0)] = t1
            sea[(1, 0)] = t2
            break
            
    # print(f'Sea is {sea}')
    
    d = max(x for x, _ in square) + 1
    for x in range(2, d):
        tile_1 = sea[(x - 1, 0)]
        tile_2 = tiles[square[(x, 0)]]
        
        for t in flips_and_rotates(tile_2):
            if east_edge(tile_1) == west_edge(t):
                sea[(x, 0)] = t
                break
                
    for y in range(1, d):
        for x in range(d):
            tile_1 = sea[(x, y - 1)]
            tile_2 = tiles[square[(x, y)]]
        
            for t in flips_and_rotates(tile_2):
                if south_edge(tile_1) == north_edge(t):
                    sea[(x, y)] = t    
            
    return sea

In [71]:
def remove_edges(orient):
    result = {}
    for k, v in orient.items():
        result[k] = pvector([line[1:-1] for line in v[1:-1]])
    return result


def create_image(trimmed):
    d = max(x for x, _ in trimmed) + 1
    n = len(trimmed[(0, 0)][0])
    result = pvector()
    
    for x in range(d):
        for i in range(n):
            result = result.append(''.join(reduce(operator.add, (trimmed[(y, x)][i] for y in range(d)))))
            
    return result


In [72]:
SEA_MONSTER = '''                  # 
#    ##    ##    ###
 #  #  #  #  #  #   '''.replace(' ', '.')

print(SEA_MONSTER)

..................#.
#....##....##....###
.#..#..#..#..#..#...


In [73]:
def find_sea_monsters(image):
    d = len(image)
    sea_monster_length = len(SEA_MONSTER.split()[0])
    sea_monster_res = [re.compile(line) for line in SEA_MONSTER.split()]
    # print(sea_monster_res)
    sea_monsters = 0
    
    for x in range(d - sea_monster_length):
        for y in range(d - 3):
            there_is_a_sea_monster = True
            for i in range(3):
#                 print(y, x, i)
#                 print(sea_monster_res[i])
#                 print(image[y + i][x:x + sea_monster_length])
#                 print(re.match(sea_monster_res[i], image[y + i][x:x + sea_monster_length]))
                if not re.match(sea_monster_res[i], image[y + i][x:x + sea_monster_length]):
                    there_is_a_sea_monster = False
                    
            if there_is_a_sea_monster:    
                sea_monsters += 1
                
    return sea_monsters

In [82]:
def part_2(tiles):
    g = generate_graph(tiles)
    #print(g)
    square = map_square(g)
    #print(square)
    orient = orient_tiles(tiles, square)
    trimmed = remove_edges(orient)
    image = create_image(trimmed)
    seamonsters = sum(
        find_sea_monsters(transform) 
        for transform in flips_and_rotates(image)
    )
    return ''.join(image).count('#') - seamonsters * SEA_MONSTER.count('#')

In [83]:
part_2(test_tiles)

273

In [84]:
part_2(tiles)

2118

AT LAST. 

I must never again get this far into the weeds with one of these problems. What a waste of time.